## Loading the Data

In [1]:
from datasets import load_dataset, concatenate_datasets

dataset = load_dataset('billingsmoore/84000-bo-en', split='train').train_test_split(test_size=0.1)
dataset.save_to_disk('ds')

README.md:   0%|          | 0.00/991 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/135620 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/122058 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/13562 [00:00<?, ? examples/s]

In [2]:
dataset

DatasetDict({
    train: Dataset({
        features: ['bo', 'en', 'source_file'],
        num_rows: 122058
    })
    test: Dataset({
        features: ['bo', 'en', 'source_file'],
        num_rows: 13562
    })
})

## Load Unfinetuned Tokenizer, Model, and Data Collator

In [3]:
from transformers import AutoTokenizer, DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained('billingsmoore/getok-v0')
model = AutoModelForSeq2SeqLM.from_pretrained("google/t5-efficient-tiny", device_map="cuda:0")
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

config.json:   0%|          | 0.00/628 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/62.3M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/62.3M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

## Preprocess Data

The dataset can now be tokenized for training.

In [4]:
def bo_en_preprocess_function(examples):

    # Prepare translation inputs and targets
    translation_inputs = ['Translate Tibetan to English: ' + example for example in examples['bo']]
    translation_targets = [example for example in examples['en']]
    
    # Tokenize translation inputs and targets
    bo_en_model_inputs = tokenizer(translation_inputs, text_target=translation_targets, 
                                         max_length=300, truncation=True, padding="max_length")
    
    
    return bo_en_model_inputs

In [5]:
def en_bo_preprocess_function(examples):

    # Prepare translation inputs and targets
    translation_inputs = ['Translate English to Tibetan: ' + example for example in examples['en']]
    translation_targets = [example for example in examples['bo']]
    
    # Tokenize translation inputs and targets
    en_bo_model_inputs = tokenizer(translation_inputs, text_target=translation_targets, 
                                         max_length=300, truncation=True, padding="max_length")
    
    
    return en_bo_model_inputs

In [6]:
bo_en_tokenized_dataset = dataset.map(bo_en_preprocess_function, batched=True)

Map:   0%|          | 0/122058 [00:00<?, ? examples/s]

Map:   0%|          | 0/13562 [00:00<?, ? examples/s]

In [7]:
en_bo_tokenized_dataset = dataset.map(en_bo_preprocess_function, batched=True)

Map:   0%|          | 0/122058 [00:00<?, ? examples/s]

Map:   0%|          | 0/13562 [00:00<?, ? examples/s]

In [8]:
from datasets import DatasetDict

tokenized_dataset = DatasetDict()

tokenized_dataset['train'] = concatenate_datasets([bo_en_tokenized_dataset['train'], en_bo_tokenized_dataset['train']])
tokenized_dataset['test'] = bo_en_tokenized_dataset['test']

## Train the Model

Finally, we can train the model. Note that the optimizer used is Adafactor. This is the optimizer that is preferred for translation tasks and for the T5 model in general. The transformers api includes a built in version of Adafactor, but I define it separately here so that we can optimize it with the 'accelerate' library.

In [9]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, Adafactor
from accelerate import Accelerator

accelerator = Accelerator()

optimizer = Adafactor(
    model.parameters(), 
    scale_parameter=True, 
    relative_step=False, 
    warmup_init=False, 
    lr=3e-4
)

model, optimizer = accelerator.prepare(model, optimizer)

In [10]:
import numpy as np
import evaluate

# Load BLEU and CHRF metrics
bleu_metric = evaluate.load("sacrebleu")
chrf_metric = evaluate.load("chrf")
ter_metric = evaluate.load("ter")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    
    # Decode predictions and labels
    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Postprocess text
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    # Compute BLEU score
    bleu_result = bleu_metric.compute(predictions=decoded_preds, references=decoded_labels)
    bleu_score = bleu_result["score"]

    # Compute CHRF score
    chrf_result = chrf_metric.compute(predictions=decoded_preds, references=decoded_labels)
    chrf_score = chrf_result["score"]

    # Compute TER score
    ter_result = ter_metric.compute(predictions=decoded_preds, references=decoded_labels)
    ter_score = ter_result["score"]

    # Return rounded results
    metrics = {
        "bleu": round(bleu_score, 4),
        "chrf": round(chrf_score, 4),
        "ter": round(ter_score, 4)
    }

    #print("Computed Metrics:", metrics)

    return metrics

In [11]:
import wandb

wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: billingsmoore. Use `wandb login --relogin` to force relogin


True

In [12]:
training_args = Seq2SeqTrainingArguments(
    output_dir=f"bidirectional",
    auto_find_batch_size=True,
    predict_with_generate=True,
    fp16=False, #check this
    push_to_hub=False,
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    num_train_epochs=3
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    tokenizer=tokenizer,
    optimizers=(optimizer, None),
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

/tmp/ipykernel_8641/4140109040.py:13: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss,Bleu,Chrf,Ter
1,0.462400,0.436817,5.295300,15.254500,103.491900
2,0.403800,0.387852,7.466500,18.154900,98.490100
3,0.391900,0.375420,8.199300,18.747400,95.325800


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=91545, training_loss=0.4509069881431917, metrics={'train_runtime': 11442.4374, 'train_samples_per_second': 64.003, 'train_steps_per_second': 8.0, 'total_flos': 9683248210329600.0, 'train_loss': 0.4509069881431917, 'epoch': 3.0})